# Run Neural Net Extension

The script consists of image processing pipeline, original network and modified network.

Output is learnt representation layers at given epoch, in form of numpy array.

In [ ]:
# Import libraries
from __future__ import print_function
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.nn.functional import sigmoid, relu
from scipy.cluster.hierarchy import dendrogram, linkage
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import the models for image transformation
from tensorflow.keras.applications import (vgg16)

vgg_model = vgg16.VGG16(weights='imagenet')

In [ ]:
#!pip install git+git://github.com/jaredwinick/img2vec-keras.git

### Load Original Dataset

In [ ]:
path = "/content/drive/My Drive/1016 Cognition Project/"

with open(path+'sem_items_animal_only.txt','r') as fid:
    names_items = np.array([l.strip() for l in fid.readlines()])
with open(path+'sem_relations_animal_only.txt','r') as fid:
    names_relations = np.array([l.strip() for l in fid.readlines()])
with open(path+'sem_attributes_animal_only.txt','r') as fid:
    names_attributes = np.array([l.strip() for l in fid.readlines()])
        
nobj = len(names_items)
nrel = len(names_relations)
nattributes = len(names_attributes)
#print('List of items:')
#print(names_items)
#print("List of relations:")
#print(names_relations)
#print("List of attributes:")
#print(names_attributes)

In [ ]:
print("number of items: ", nobj)
print("number of attributes: ", nattributes)

number of items:  77
number of attributes:  239


In [ ]:
D = np.loadtxt(path+'sem_data_animal_only.txt')
input_pats = D[:,:nobj+nrel]
input_pats = torch.tensor(input_pats,dtype=torch.float)
output_pats = D[:,nobj+nrel:]
output_pats = torch.tensor(output_pats,dtype=torch.float)
N = input_pats.shape[0] # number of training patterns
input_v = input_pats[0,:].numpy().astype('bool')
output_v = output_pats[0,:].numpy().astype('bool')
print('Example input pattern:')
print(input_v.astype('int'))
print('Example output pattern:')
print(output_v.astype('int'))
print("")
print("Which encodes...")
print('Item ',end='')
print(names_items[input_v[:nobj]])
print('Relation ',end='')
print(names_relations[input_v[nobj:]])
print('Attributes ',end='')
print(names_attributes[output_v])

Example input pattern:
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0]
Example output pattern:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]

Which encodes...
Item ['African elephant']
Relation ['can']
Attributes ['view']


In [ ]:
for i in range(N):
    input_v = input_pats[i,:].numpy().astype('bool')
    output_v = output_pats[i,:].numpy().astype('bool')
    #print(names_items[input_v[:nobj]],names_relations[input_v[nobj:]],names_attributes[output_v])

### 2. Get Association Layer from Image, then concatenate

In [ ]:
names_items_processed = [name.lower() for name in names_items]
names_items_processed = [name.replace(' ','_') for name in names_items_processed]

In [ ]:
from img2vec_keras import Img2Vec
img2vec = Img2Vec()

In [ ]:
def get_vec(input_pat_vec):
  # Binary input vector pattern, same as "Example input pattern" above
  index = input_pat_vec.index(1)
  item = names_items_processed[index] # name of the animal
  image_path = path+'images/'+item+'.jpg'
  image_vec = img2vec.get_vec(image_path)
  return image_vec

In [ ]:
vector_list = []

for item_ind in range(len(input_pats)):
  input_v = input_pats[item_ind,:].numpy().astype('bool')
  input_pat_vec = list(input_v.astype('int'))
  image_vec = get_vec(input_pat_vec)
  vector_list.append(image_vec)

In [ ]:
# Compute similarity to each of the category group (by taking average of 4 pictures of each category group)

category_names = ['amphibians', 'birds', 'fish', 'invertebrates', 'mammals', 'reptiles']

cat_vectors = {}
for cat_name in category_names:
  temp = []
  temp.append(img2vec.get_vec(path+'images/'+cat_name+'1.jpg'))
  temp.append(img2vec.get_vec(path+'images/'+cat_name+'2.jpg'))
  temp.append(img2vec.get_vec(path+'images/'+cat_name+'3.jpg'))
  temp.append(img2vec.get_vec(path+'images/'+cat_name+'4.jpg'))
  cat_vectors[cat_name] = temp

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def get_category_similarity(item_index, cat_name):
  cat_vec1 = cat_vectors[cat_name][0]
  cat_vec2 = cat_vectors[cat_name][1]
  cat_vec3 = cat_vectors[cat_name][2]
  cat_vec4 = cat_vectors[cat_name][3]
  item_vec = vector_list[item_index]
  X = np.stack([item_vec, cat_vec1, cat_vec2, cat_vec3, cat_vec4])
  Y = X
  sim_matrix = cosine_similarity(X,Y)
  sim_score = sim_matrix[0][1:].mean()
  return sim_score

In [ ]:
# Compute and store similarity score with each of 6 category
sim_results = {}

for cat_i in category_names:
  temp_storage = []
  for item_ind in range(len(vector_list)):
    item_vec = vector_list[item_ind]
    temp_storage.append(get_category_similarity(item_ind, cat_i))
  sim_results[cat_i] = temp_storage

In [ ]:
# Transform the category similarities and concatenate to original input

association_raw = []
for cat in sim_results.keys():
  association_raw.append(sim_results[cat])

association_raw = torch.tensor(association_raw)
association = torch.transpose(association_raw, 0, 1)
input_pats_new = torch.cat((input_pats, association), 1)

#input_pats_new.size()

### 3. Original NN and Training

In [ ]:
class Net(nn.Module):
    def __init__(self, rep_size, hidden_size):
        super(Net, self).__init__()
        # Input
        #  rep_size : number of hidden units in "Representation Layer"
        #  hidden_Size : number of hidden units in "Hidden Layer"
        #
        # TODO : YOUR CODE GOES HERE
        self.fc_items_to_representation = nn.Linear(nobj, rep_size)
        self.fc_representation_relation_to_hidden = nn.Linear(rep_size + nrel, hidden_size)
        self.fc_hidden_to_attribute = nn.Linear(hidden_size, nattributes)
        
        #raise Exception('Replace with your code.')

    def forward(self, x):
        # Defines forward pass for the network on input patterns x
        #
        # Input can take these two forms:
        #
        #   x: [nobj+nrel 1D Tensor], which is a single input pattern as a 1D tensor
        #      (containing both object and relation 1-hot identifier) (batch size is B=1)
        #   OR
        #   x : [B x (nobj+nrel) Tensor], which is a batch of B input patterns (one for each row)
        #
        # Output
        #   output [B x nattribute Tensor], which is the output pattern for each input pattern B on the Attribute Layer
        #   hidden [B x hidden_size Tensor], which are activations in the Hidden Layer
        #   rep [B x rep_size Tensor], which are the activations in the Representation LAyer
        x = x.view(-1,nobj+nrel) # reshape as size [B x (nobj+nrel) Tensor] if B=1
        x_item = x[:,:nobj] # input to Item Layer [B x nobj Tensor]
        x_rel = x[:,nobj:] # input to Relation Layer [B x nrel Tensor]
        # TODO : YOUR CODE GOES HERE
        # ----
        netinput_rep = self.fc_items_to_representation(x_item)
        rep = relu(netinput_rep)
        rep_rel = torch.cat((rep, x_rel),1) 
        netinput_hidden = self.fc_representation_relation_to_hidden(rep_rel)
        hidden = relu(netinput_hidden)
        netinput_attribute = self.fc_hidden_to_attribute(hidden)
        output = sigmoid(netinput_attribute)
        
        #raise Exception('Replace with your code.')
        # -----
        return output, hidden, rep

In [ ]:
def train(mynet,epoch_count,nepochs_additional=5000):
    # Input
    #  mynet : Net class object
    #  epoch_count : (scalar) how many epochs have been completed so far
    #  nepochs_additional : (scalar) how many more epochs we want to run
    mynet.train()
    for e in range(nepochs_additional): # for each epoch
        error_epoch = 0.
        perm = np.random.permutation(N)
        for p in perm: # iterate through input patterns in random order
            mynet.zero_grad() # reset gradient
            output, hidden, rep = mynet(input_pats[p,:]) # forward pass
            target = output_pats[p,:] 
            #print output and compare with target
            #if e % 50 == 0:
                #print("target: ",names_attributes[target.numpy().astype('bool')])
                #print("output: ",names_attributes[np.where(output.detach().numpy()>0.5,1,0).astype('bool').flatten()])
            loss = criterion(output, target) # compute loss
            loss.backward() # compute gradient 
            optimizer.step() # update network parameters
            error_epoch += loss.item()
        error_epoch = error_epoch / float(N)        
        if e % 50 == 0:
            print('epoch ' + str(epoch_count+e) + ' loss ' + str(round(error_epoch,3)))
    return epoch_count + nepochs_additional

In [ ]:
def get_rep(net):
    # Extract the hidden activations on the Representation Layer for each item
    # 
    # Input
    #  net : Net class object
    #
    # Output
    #  rep : [nitem x rep_size numpy array], where each row is an item
    input_clean = torch.zeros(nobj,nobj+nrel)
    for idx,name in enumerate(names_items):
        input_clean[idx,idx] = 1. # 1-hot encoding of each object (while Relation Layer doesn't matter)
    output, hidden, rep = mynet(input_clean)
    return rep.detach().numpy()

def plot_rep(rep1,rep2,rep3,names):
    #  Compares Representation Layer activations of Items at three different times points in learning (rep1, rep2, rep3)
    #  using bar graphs
    # 
    #  Each rep1, rep2, rep3 is a [nitem x rep_size numpy array]
    #  names : [nitem list] of item names
    #
    nepochs_list = [nepochs_phase1,nepochs_phase2,nepochs_phase3]
    nrows = nobj
    R = np.dstack((rep1,rep2,rep3))    
    mx = R.max()
    mn = R.min()
    depth = R.shape[2]
    count = 1
    plt.figure(1,figsize=(4.2,360))
    for i in range(nrows):
        for d in range(R.shape[2]):
            plt.subplot(nrows, depth, count)
            rep = R[i,:,d]
            plt.bar(range(rep.size),rep)
            plt.ylim([mn,mx])
            plt.xticks([])
            plt.yticks([])        
            if d==0:
                plt.ylabel(names[i])
            if i==0:
                plt.title("epoch " + str(nepochs_list[d]))
            count += 1
    plt.show()

def plot_dendo(rep1,rep2,rep3,names):
    #  Compares Representation Layer activations of Items at three different times points in learning (rep1, rep2, rep3)
    #  using hierarchical clustering
    # 
    #  Each rep1, rep2, rep3 is a [nitem x rep_size numpy array]
    #  names : [nitem list] of item names
    #
    nepochs_list = [nepochs_phase1,nepochs_phase2,nepochs_phase3]
    linked1 = linkage(rep1,'single')
    linked2 = linkage(rep2,'single')
    linked3 = linkage(rep3,'single')
    mx = np.dstack((linked1[:,2],linked2[:,2],linked3[:,2])).max()+0.1    
    plt.figure(2,figsize=(24, 20))
    plt.subplot(3,1,1)    
    dendrogram(linked1, labels=names, color_threshold=0)
    plt.ylim([0,mx])
    plt.title('Hierarchical clustering; ' + "epoch " + str(nepochs_list[0]))
    plt.ylabel('Euclidean distance')
    plt.subplot(3,1,2)
    plt.title("epoch " + str(nepochs_list[1]))
    dendrogram(linked2, labels=names, color_threshold=0)
    plt.ylim([0,mx])
    plt.subplot(3,1,3)
    plt.title("epoch " + str(nepochs_list[2]))
    dendrogram(linked3, labels=names, color_threshold=0)
    plt.ylim([0,mx])
    plt.show()

In [ ]:
learning_rate = 0.1
criterion = nn.MSELoss() # mean squared error loss function
mynet = Net(rep_size=8,hidden_size=15)
optimizer = torch.optim.SGD(mynet.parameters(), lr=learning_rate) # stochastic gradient descent

nepochs_phase1 = 500
nepochs_phase2 = 1000
nepochs_phase3 = 2500
epoch_count = 0
epoch_count = train(mynet,epoch_count,nepochs_additional=nepochs_phase1)
rep1 = get_rep(mynet)
epoch_count = train(mynet,epoch_count,nepochs_additional=nepochs_phase2-nepochs_phase1)
rep2 = get_rep(mynet)
epoch_count = train(mynet,epoch_count,nepochs_additional=nepochs_phase3-nepochs_phase2)
rep3 = get_rep(mynet)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([239])) that is different to the input size (torch.Size([1, 239])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 0 loss 0.25
epoch 50 loss 0.017
epoch 100 loss 0.015
epoch 150 loss 0.013
epoch 200 loss 0.013
epoch 250 loss 0.012
epoch 300 loss 0.012
epoch 350 loss 0.012
epoch 400 loss 0.012
epoch 450 loss 0.012
epoch 500 loss 0.012
epoch 550 loss 0.012
epoch 600 loss 0.012
epoch 650 loss 0.012
epoch 700 loss 0.011
epoch 750 loss 0.011
epoch 800 loss 0.011
epoch 850 loss 0.011
epoch 900 loss 0.011
epoch 950 loss 0.011
epoch 1000 loss 0.011
epoch 1050 loss 0.011
epoch 1100 loss 0.011
epoch 1150 loss 0.011
epoch 1200 loss 0.011
epoch 1250 loss 0.011
epoch 1300 loss 0.011
epoch 1350 loss 0.011
epoch 1400 loss 0.01
epoch 1450 loss 0.01
epoch 1500 loss 0.01
epoch 1550 loss 0.01
epoch 1600 loss 0.01
epoch 1650 loss 0.01
epoch 1700 loss 0.01
epoch 1750 loss 0.01
epoch 1800 loss 0.01
epoch 1850 loss 0.01
epoch 1900 loss 0.01
epoch 1950 loss 0.01
epoch 2000 loss 0.01
epoch 2050 loss 0.01
epoch 2100 loss 0.01
epoch 2150 loss 0.01
epoch 2200 loss 0.01
epoch 2250 loss 0.009
epoch 2300 loss 0.009
epoch 2

In [ ]:
#plot_rep(rep1,rep2,rep3,names_items)
#plot_dendo(rep1,rep2,rep3,names_items)

In [ ]:
rep3.shape

(77, 8)

In [ ]:
np.save(path+"rep_array_animal_orig.npy", rep3)

### 4. Modified NN and Training (Association Layer enters Hidden Layer directly)

In [ ]:
nassociation = 6

print('# Objects:', nobj)
print('# Relations:', nrel)
print('# Attributes:', nattributes)
print('# Associations:', nassociation)

# Objects: 77
# Relations: 5
# Attributes: 239
# Associations: 6


In [ ]:
class Net(nn.Module):
    def __init__(self, rep_size, hidden_size):
        super(Net, self).__init__()
        # Input
        # rep_size : number of hidden units in "Representation Layer"
        # hidden_Size : number of hidden units in "Hidden Layer"
        self.rep_size = rep_size
        self.hidden_size = hidden_size
        # function for linear transformation:
        #   fc1: from item to representation
        #   fc2: from representation+relation+association to hidden
        #   fc3: from hidden layer to output 
        self.fc1 = nn.Linear(nobj, rep_size)
        self.fc2 = nn.Linear(rep_size+nrel+nassociation, hidden_size)
        self.fc3 = nn.Linear(hidden_size, nattributes)

    def forward(self, x):
        # Defines forward pass for the network on input patterns x
        #
        # Input can take these two forms:
        #
        #   x: [nobj+nrel 1D Tensor], which is a single input pattern as a 1D tensor
        #      (containing both object and relation 1-hot identifier) (batch size is B=1)
        #   OR
        #   x : [B x (nobj+nrel) Tensor], which is a batch of B input patterns (one for each row)
        #
        # Output
        #   output [B x nattribute Tensor], which is the output pattern for each input pattern B on the Attribute Layer
        #   hidden [B x hidden_size Tensor], which are activations in the Hidden Layer
        #   rep [B x rep_size Tensor], which are the activations in the Representation LAyer
        
        x = x.view(-1,nobj+nrel+nassociation) # reshape as size [B x (nobj+nrel) Tensor] if B=1
        x_item = x[:,:nobj] # input to Item Layer [B x nobj Tensor]
        x_rel = x[:,nobj:nobj+nrel] # input to Relation Layer [B x nrel Tensor]
        x_association = x[:,nobj+nrel:] # input to Association Layer [B x nassociation Tensor]
        
        rep = relu(self.fc1(x_item)) # representation layer tensor
        rep_and_rel_and_association = torch.cat((rep.view(rep.size(0), -1), 
                                                 x_rel.view(x_rel.size(0), -1), 
                                                 x_association.view(x_association.size(0), -1)), dim=1)
        hidden = relu(self.fc2(rep_and_rel_and_association))
        output = sigmoid(self.fc3(hidden))
    
        return output, hidden, rep

In [ ]:
# Update train function: replace input_pats with input_pats_new

def train(mynet,epoch_count,nepochs_additional=5000):
    # Input
    #  mynet : Net class object
    #  epoch_count : (scalar) how many epochs have been completed so far
    #  nepochs_additional : (scalar) how many more epochs we want to run
    mynet.train()
    for e in range(nepochs_additional): # for each epoch
        error_epoch = 0.
        perm = np.random.permutation(N)
        for p in perm: # iterate through input patterns in random order
            mynet.zero_grad() # reset gradient
            output, hidden, rep = mynet(input_pats_new[p,:]) # forward pass
            target = output_pats[p,:] 
            loss = criterion(output, target) # compute loss
            loss.backward() # compute gradient 
            optimizer.step() # update network parameters
            error_epoch += loss.item()
        error_epoch = error_epoch / float(N)        
        if e % 50 == 0:
            print('epoch ' + str(epoch_count+e) + ' loss ' + str(round(error_epoch,3)))
    return epoch_count + nepochs_additional

In [ ]:
# Change dimension of input for get_rep

def get_rep(net):
    '''
    Extract the hidden activations on the Representation Layer for each item
    Input
       net : Net class object
    # Output
       rep : [nitem x rep_size numpy array], where each row is an item
    '''
    input_clean = torch.zeros(nobj,nobj+nrel+nassociation)
    for idx,name in enumerate(names_items):
        input_clean[idx,idx] = 1. # 1-hot encoding of each object (while Relation Layer doesn't matter)
    output, hidden, rep = mynet(input_clean)
    return rep.detach().numpy()

In [ ]:
learning_rate = 0.1
criterion = nn.MSELoss() # mean squared error loss function
mynet = Net(rep_size=8,hidden_size=15)
optimizer = torch.optim.SGD(mynet.parameters(), lr=learning_rate) # stochastic gradient descent

nepochs_phase1 = 500
nepochs_phase2 = 1000
nepochs_phase3 = 2500
epoch_count = 0
epoch_count = train(mynet,epoch_count,nepochs_additional=nepochs_phase1)
rep1 = get_rep(mynet)
epoch_count = train(mynet,epoch_count,nepochs_additional=nepochs_phase2-nepochs_phase1)
rep2 = get_rep(mynet)
epoch_count = train(mynet,epoch_count,nepochs_additional=nepochs_phase3-nepochs_phase2)
rep3 = get_rep(mynet)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([239])) that is different to the input size (torch.Size([1, 239])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 0 loss 0.243
epoch 50 loss 0.017
epoch 100 loss 0.015
epoch 150 loss 0.013
epoch 200 loss 0.012
epoch 250 loss 0.012
epoch 300 loss 0.012
epoch 350 loss 0.012
epoch 400 loss 0.012
epoch 450 loss 0.012
epoch 500 loss 0.012
epoch 550 loss 0.012
epoch 600 loss 0.012
epoch 650 loss 0.012
epoch 700 loss 0.011
epoch 750 loss 0.011
epoch 800 loss 0.011
epoch 850 loss 0.011
epoch 900 loss 0.011
epoch 950 loss 0.011
epoch 1000 loss 0.011
epoch 1050 loss 0.011
epoch 1100 loss 0.011
epoch 1150 loss 0.011
epoch 1200 loss 0.011
epoch 1250 loss 0.011
epoch 1300 loss 0.01
epoch 1350 loss 0.01
epoch 1400 loss 0.01
epoch 1450 loss 0.01
epoch 1500 loss 0.01
epoch 1550 loss 0.01
epoch 1600 loss 0.01
epoch 1650 loss 0.01
epoch 1700 loss 0.01
epoch 1750 loss 0.01
epoch 1800 loss 0.01
epoch 1850 loss 0.01
epoch 1900 loss 0.01
epoch 1950 loss 0.01
epoch 2000 loss 0.009
epoch 2050 loss 0.009
epoch 2100 loss 0.009
epoch 2150 loss 0.009
epoch 2200 loss 0.009
epoch 2250 loss 0.009
epoch 2300 loss 0.009
epo

In [ ]:
#plot_rep(rep1,rep2,rep3,names_items)
#plot_dendo(rep1,rep2,rep3,names_items)

np.save(path+"rep_array_animal_association.npy", rep3)

### 5. Adjust image vector weights by *10


In [ ]:
association_raw = []
for cat in sim_results.keys():
  orig_vec = sim_results[cat]
  weighted_vec = [float(10 * ele) for ele in orig_vec]
  association_raw.append(weighted_vec)

association_raw = torch.tensor(association_raw)
association = torch.transpose(association_raw, 0, 1)
input_pats_new = torch.cat((input_pats, association), 1)

In [ ]:
# Update train function: replace input_pats with input_pats_new

def train(mynet,epoch_count,nepochs_additional=5000):
    # Input
    #  mynet : Net class object
    #  epoch_count : (scalar) how many epochs have been completed so far
    #  nepochs_additional : (scalar) how many more epochs we want to run
    mynet.train()
    for e in range(nepochs_additional): # for each epoch
        error_epoch = 0.
        perm = np.random.permutation(N)
        for p in perm: # iterate through input patterns in random order
            mynet.zero_grad() # reset gradient
            output, hidden, rep = mynet(input_pats_new[p,:]) # forward pass
            target = output_pats[p,:] 
            loss = criterion(output, target) # compute loss
            loss.backward() # compute gradient 
            optimizer.step() # update network parameters
            error_epoch += loss.item()
        error_epoch = error_epoch / float(N)        
        if e % 50 == 0:
            print('epoch ' + str(epoch_count+e) + ' loss ' + str(round(error_epoch,3)))
    return epoch_count + nepochs_additional

In [ ]:
learning_rate = 0.1
criterion = nn.MSELoss() # mean squared error loss function
mynet = Net(rep_size=8,hidden_size=15)
optimizer = torch.optim.SGD(mynet.parameters(), lr=learning_rate) # stochastic gradient descent

nepochs_phase1 = 500
nepochs_phase2 = 1000
nepochs_phase3 = 2500
epoch_count = 0
epoch_count = train(mynet,epoch_count,nepochs_additional=nepochs_phase1)
rep1 = get_rep(mynet)
epoch_count = train(mynet,epoch_count,nepochs_additional=nepochs_phase2-nepochs_phase1)
rep2 = get_rep(mynet)
epoch_count = train(mynet,epoch_count,nepochs_additional=nepochs_phase3-nepochs_phase2)
rep3 = get_rep(mynet)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([239])) that is different to the input size (torch.Size([1, 239])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 0 loss 0.2
epoch 50 loss 0.017
epoch 100 loss 0.016
epoch 150 loss 0.015
epoch 200 loss 0.013
epoch 250 loss 0.013
epoch 300 loss 0.012
epoch 350 loss 0.012
epoch 400 loss 0.012
epoch 450 loss 0.012
epoch 500 loss 0.012
epoch 550 loss 0.012
epoch 600 loss 0.012
epoch 650 loss 0.011
epoch 700 loss 0.011
epoch 750 loss 0.011
epoch 800 loss 0.011
epoch 850 loss 0.011
epoch 900 loss 0.011
epoch 950 loss 0.011
epoch 1000 loss 0.011
epoch 1050 loss 0.011
epoch 1100 loss 0.011
epoch 1150 loss 0.011
epoch 1200 loss 0.011
epoch 1250 loss 0.01
epoch 1300 loss 0.01
epoch 1350 loss 0.01
epoch 1400 loss 0.01
epoch 1450 loss 0.01
epoch 1500 loss 0.01
epoch 1550 loss 0.01
epoch 1600 loss 0.01
epoch 1650 loss 0.01
epoch 1700 loss 0.009
epoch 1750 loss 0.009
epoch 1800 loss 0.009
epoch 1850 loss 0.009
epoch 1900 loss 0.009
epoch 1950 loss 0.009
epoch 2000 loss 0.009
epoch 2050 loss 0.009
epoch 2100 loss 0.009
epoch 2150 loss 0.009
epoch 2200 loss 0.009
epoch 2250 loss 0.009
epoch 2300 loss 0.009


In [ ]:
np.save(path+"rep_array_animal_weight10.npy", rep3)